In [ ]:
import numpy
import pygetm

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

In [ ]:
# Set up rectangular domain with outer points masked
domain = pygetm.domain.Domain.create_cartesian(500.*numpy.arange(100), 500.*numpy.arange(30), 1, lat=0, H=50)
sim = pygetm.Simulation(domain, runtype=1, advection_scheme=1)

# Idealized surface forcing
tausx = domain.U.array(fill=0.01)
tausy = domain.V.array(fill=0.)
sp = domain.T.array(fill=0.)

# Time
timestep = 10.
ntime = int(3600. // timestep)

# Set up velocity plot
nsample = 2
fig, ax = matplotlib.pyplot.subplots()
pc = ax.pcolormesh(domain.T.xi, domain.T.yi, domain.T.z, vmin=-0.001, vmax=0.001)
cb = fig.colorbar(pc)
cb.set_label('elevation (m)')
u, v = sim.U.interp(domain.T) / domain.T.D, sim.V.interp(domain.T) / domain.T.D
Q = ax.quiver(domain.T.x[::nsample, ::nsample], domain.T.y[::nsample, ::nsample], u[::nsample, ::nsample], v[::nsample, ::nsample], scale=.01)
title = ax.set_title('time: 0 s')

In [ ]:
plotting_interval = 5
times = timestep * numpy.arange(ntime)
Us = numpy.empty_like(times)
for istep, time in enumerate(times):
    Us[istep] = sim.U[...].mean()
    sim.update_surface_pressure_gradient(domain.T.z, sp)
    sim.uv_momentum_2d(timestep, tausx, tausy, sim.dpdx, sim.dpdy)
    sim.update_sealevel(timestep, sim.U, sim.V)
    sim.update_depth()

    # Update velocity plot
    if istep % plotting_interval == 0:
        u, v = sim.U.interp(domain.T) / domain.T.D, sim.V.interp(domain.T) / domain.T.D
        Q.set_UVC(u[::nsample, ::nsample], v[::nsample, ::nsample])
        title.set_text('time: %s s' % time)
        pc.set_array(domain.T.z[...].ravel())
        fig.canvas.draw()

In [ ]:
fig, ax = matplotlib.pyplot.subplots()
ax.plot(times, Us)
ax.set_title('mean U')
ax.grid()